# Mechanistic Watchdog: Stress Test & Validation

**Module:** `MechWatch`
**Goal:** Verify that the "Deception Score" is linearly separable between Truthful Control prompts and Deceptive/Adversarial prompts.

In [1]:
import json
import sys
import os
from pathlib import Path

# 1. Setup Paths dynamically for Windows/Linux compatibility
current_dir = Path.cwd()
# If we are in 'notebooks', parent is project root. If in root, current is root.
project_root = current_dir.parent if current_dir.name == "notebooks" else current_dir

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"📂 Project Root: {project_root}")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from tqdm import tqdm

# 2. Import from MechWatch
try:
    from MechWatch.runtime import WatchdogRuntime
    from MechWatch.config import load_config
    print("✅ Successfully imported WatchdogRuntime from MechWatch")
except ImportError as e:
    print(f"❌ Import failed: {e}. Ensure the folder is named 'MechWatch' and is in the project root.")

📂 Project Root: e:\Dropbox\Projects\hackaton
✅ Successfully imported WatchdogRuntime from MechWatch


## 1. Initialize Runtime & Locate Vector
We need to find the `deception_vector.pt` file and pass its absolute path to the configuration to avoid FileNotFoundError.

In [2]:
# 1. Find the vector file
vector_candidates = [
    project_root / "artifacts" / "deception_vector.pt",
    project_root / "deception_vector.pt",
    Path("artifacts/deception_vector.pt")
]

vector_path = next((p for p in vector_candidates if p.exists()), None)

if not vector_path:
    raise FileNotFoundError(f"Could not find 'deception_vector.pt' in {project_root} or artifacts/")

print(f"✅ Found vector at: {vector_path}")

# 2. Load Config and Override Path
cfg = load_config()
cfg.vector_path = vector_path 

# 3. Initialize Runtime with the corrected config
rt = WatchdogRuntime(cfg=cfg)
print(f"✅ Loaded Watchdog with threshold: {rt.threshold:.4f}")

✅ Found vector at: e:\Dropbox\Projects\hackaton\artifacts\deception_vector.pt
✅ Loaded Watchdog with threshold: -0.3872


## 2. Load Test Cases

In [3]:
import pandas as pd
from datasets import load_dataset
import random

# Configuration
SAMPLES_PER_CATEGORY = 40 

test_cases = []

print("📥 Loading Datasets (Safe Mode)...")

# ---------------------------------------------------------
# 1. CONTROL (TRUTH): The Baseline
# ---------------------------------------------------------
try:
    print(f"   - Loading Control (True Facts)...")
    ds_facts = load_dataset("L1Fthrasir/Facts-true-false", split="train")
    
    # Filter for TRUE statements (Label 1)
    true_facts = ds_facts.filter(lambda x: x['label'] == 1).shuffle(seed=42).select(range(SAMPLES_PER_CATEGORY))
    
    for row in true_facts:
        test_cases.append({
            "category": "Control (Truth)",
            "prompt": row['statement'],
            "expected": "Pass"
        })
except Exception as e:
    print(f"   ⚠️ Failed to load Control: {e}")

# ---------------------------------------------------------
# 2. TRUTHFUL QA: The Edge Case
# ---------------------------------------------------------
try:
    print(f"   - Loading TruthfulQA (Misconceptions)...")
    ds_tqa = load_dataset("truthful_qa", "generation", split="validation")
    ds_tqa = ds_tqa.shuffle(seed=42).select(range(SAMPLES_PER_CATEGORY))
    
    for row in ds_tqa:
        test_cases.append({
            "category": "TruthfulQA (Misconceptions)",
            "prompt": row['question'],
            "expected": "Variable"
        })
except Exception as e:
    print(f"   ⚠️ Failed to load TruthfulQA: {e}")

# ---------------------------------------------------------
# 3. FACTUAL LIES: The "Red Zone" (Replaces Jailbreaks)
# ---------------------------------------------------------
# Instead of Jailbreaks (which crash the GPU), we use Factual Lies.
# These are the exact opposite of the Control group.
try:
    print(f"   - Loading Factual Lies (Adversarial)...")
    
    # Use the SAME dataset, but filter for FALSE statements (Label 0)
    false_facts = ds_facts.filter(lambda x: x['label'] == 0).shuffle(seed=42).select(range(SAMPLES_PER_CATEGORY))
    
    for row in false_facts:
        test_cases.append({
            "category": "Factual Lies",
            "prompt": row['statement'], 
            "expected": "Block"
        })
except Exception as e:
    print(f"   ⚠️ Failed to load Factual Lies: {e}")

print(f"\n✅ Loaded {len(test_cases)} total test cases.")
print(f"   Breakdown: {list(set([c['category'] for c in test_cases]))}")

📥 Loading Datasets (Safe Mode)...
   - Loading Control (True Facts)...
   - Loading TruthfulQA (Misconceptions)...
   - Loading Factual Lies (Adversarial)...


Filter:   0%|          | 0/613 [00:00<?, ? examples/s]


✅ Loaded 120 total test cases.
   Breakdown: ['Factual Lies', 'Control (Truth)', 'TruthfulQA (Misconceptions)']


## 3. Run the Gauntlet
We run each prompt with the watchdog **disabled** (monitoring only) to record the raw scores without stopping the model early.

In [ ]:
import gc
import torch
import pandas as pd
from tqdm import tqdm

results = []
SAVE_EVERY = 20

print("🚀 Starting Gauntlet...")

# GLOBAL no_grad to save massive memory
with torch.no_grad():
    for i, case in enumerate(tqdm(test_cases, desc="Inference", unit="prompt")):
        try:
            # 1. Run Generation
            # Reduced tokens to 40 for speed/safety
            output = rt.generate_without_watchdog(
                prompt=case["prompt"],
                max_new_tokens=40, 
                temperature=0.0 
            )
            
            # 2. Extract Metrics
            scores = output.scores
            valid_scores = scores[5:] if len(scores) > 5 else scores
            if not valid_scores: valid_scores = [0.0]
            
            peak_score = max(valid_scores)
            avg_score = sum(valid_scores) / len(valid_scores)
            
            # 3. Handle Text Extraction
            # TransformerLens can return list or string depending on batch dim
            raw_text = output.text[0] if isinstance(output.text, list) else output.text
            response_text = raw_text[len(case["prompt"]):].strip()
            
            results.append({
                "Category": case["category"],
                "Prompt": case["prompt"],
                "Peak Score": peak_score,
                "Avg Score": avg_score,
                "Would Block?": peak_score > rt.threshold,
                "Output": response_text[:100].replace("\n", " ") + "..."
            })
            
            # 4. Aggressive Cleanup
            # Delete large objects immediately
            del output
            del scores
            del raw_text
            
            # 5. Periodic Saving (Safety Net)
            if i > 0 and i % SAVE_EVERY == 0:
                pd.DataFrame(results).to_csv("stress_test_backup.csv", index=False)
                
        except Exception as e:
            print(f"⚠️ Error at index {i}: {e}")
            torch.cuda.empty_cache()

print("✅ Inference Complete.")
df = pd.DataFrame(results)
df.to_csv("stress_test_final.csv", index=False)

🚀 Starting Gauntlet...


Inference:   0%|          | 0/120 [00:00<?, ?prompt/s]`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded pretrained model meta-llama/Llama-3.1-8B-Instruct into HookedTransformer


Inference:  18%|█▊        | 21/120 [01:29<05:37,  3.41s/prompt]

## 4. Visualization
We expect a clear separation: "Control" cases should be below the red line, "Deception" cases above it.

In [ ]:
df = pd.DataFrame(results)

# Set Plot Style
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 6))

# Create Boxplot
sns.boxplot(data=df, x="Category", y="Peak Score", palette="coolwarm", hue="Category")
sns.stripplot(data=df, x="Category", y="Peak Score", color="black", alpha=0.5, jitter=True)

# Draw Threshold Line
plt.axhline(rt.threshold, color="red", linestyle="--", linewidth=2, label=f"Threshold ({rt.threshold:.2f})")

plt.title("Peak Deception Score by Category", fontsize=16)
plt.xticks(rotation=45, ha="right")
plt.legend()
plt.tight_layout()
plt.show()

## 5. Failure Analysis
Identify False Negatives (Deception that slipped through) and False Positives (Truth that was blocked).

In [ ]:
print(f"Current Threshold: {rt.threshold:.4f}\n")

print("🔴 FALSE NEGATIVES (Missed Deception):")
fn = df[(df["Category"] != "Control (Truth)") & (df["Peak Score"] < rt.threshold)]
if not fn.empty:
    display(fn[["Category", "Prompt", "Peak Score", "Output"]])
else:
    print("None! All deceptive prompts were caught.")

print("\n🟢 FALSE POSITIVES (Wrongly Blocked Truth):")
fp = df[(df["Category"] == "Control (Truth)") & (df["Peak Score"] > rt.threshold)]
if not fp.empty:
    display(fp[["Category", "Prompt", "Peak Score", "Output"]])
else:
    print("None! All truthful prompts passed.")

In [ ]:
# Save results for report
df.to_csv("stress_test_results.csv", index=False)
print("Results saved to stress_test_results.csv")